In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker

import os
import urllib
import re
import random
import json
from typing import List, Dict, Optional, Any, Tuple
import glob

import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
import pickle
from torch.nn import functional as F

from collections import OrderedDict, Counter

In [2]:
import tokenizers
#from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.pre_tokenizers import Punctuation

from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, Strip, Replace, Sequence
from tokenizers.trainers import UnigramTrainer

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(1)

In [4]:
HEMINGWAY_DATA = './data/hemingway'
!mkdir -p $HEMINGWAY_DATA


In [5]:
#!pip install sacrebleu
!pip show sacrebleu

Name: sacrebleu
Version: 1.5.1
Summary: Hassle-free computation of shareable, comparable, and reproducible BLEU, chrF, and TER scores
Home-page: https://github.com/mjpost/sacrebleu
Author: Matt Post
Author-email: post@cs.jhu.edu
License: Apache License 2.0
Location: /home/guntis/anaconda3/envs/tw131/lib/python3.8/site-packages
Requires: portalocker
Required-by: 


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

#Text preprocessing

In [7]:
num_bpe_merges = 10000
vocab_size = 5500
joint_vocab_size = 2*vocab_size

!echo BPE_ops=$num_bpe_merges vocab_size=$vocab_size joint_vocab_size=$joint_vocab_size

BPE_ops=10000 vocab_size=5500 joint_vocab_size=11000


In [8]:
#!pip install subword-nmt
!pip show subword-nmt

Name: subword-nmt
Version: 0.3.7
Summary: Unsupervised Word Segmentation for Neural Machine Translation and Text Generation
Home-page: https://github.com/rsennrich/subword-nmt
Author: Rico Sennrich
Author-email: None
License: MIT
Location: /home/guntis/anaconda3/envs/tw131/lib/python3.8/site-packages
Requires: 
Required-by: 


In [9]:
# Read Hemingway texts from URL. There are Hemingway's "A Farewell to arms"
text_en = urllib.request.urlopen('http://www.ltn.lv/~guntis/translation_dataset/dataset_en_small.txt').read().decode("utf-8", "ignore")
text_lv = urllib.request.urlopen('http://www.ltn.lv/~guntis/translation_dataset/dataset_lv_small.txt').read().decode("utf-8-sig", "ignore")

HEMINGWAY_SRC_EN = f'{HEMINGWAY_DATA}/hemingway.en.txt'
HEMINGWAY_SRC_LV = f'{HEMINGWAY_DATA}/hemingway.lv.txt'

with open(HEMINGWAY_SRC_EN, 'w') as f:
    f.write(text_en)

with open(HEMINGWAY_SRC_LV, 'w') as f:
    f.write(text_lv)

In [10]:
# !git clone https://github.com/moses-smt/mosesdecoder.git

In [11]:
# Normalize and tokenize texts

!cat $HEMINGWAY_SRC_EN | mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l en \
  | mosesdecoder/scripts/tokenizer/tokenizer.perl -a -l en > $HEMINGWAY_DATA/hemingway.en.tok.txt

!cat $HEMINGWAY_SRC_LV | mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l lv \
  | mosesdecoder/scripts/tokenizer/tokenizer.perl -a -l lv > $HEMINGWAY_DATA/hemingway.lv.tok.txt

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: lv
Number of threads: 1


In [12]:
# # Normalize and tokenize texts

# #!cat hemingway.en.txt | mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l en \
# !cat hemingway.en.txt \
#   | mosesdecoder/scripts/tokenizer/tokenizer.perl -a -l en > hemingway.en.tok.txt

# # !cat hemingway.lv.txt | mosesdecoder/scripts/tokenizer/normalize-punctuation.perl -l lv \
# !cat hemingway.lv.txt \
#   | mosesdecoder/scripts/tokenizer/tokenizer.perl -a -l lv > hemingway.lv.tok.txt

In [13]:
!mosesdecoder/scripts/recaser/train-truecaser.perl -corpus $HEMINGWAY_DATA/hemingway.en.tok.txt -model $HEMINGWAY_DATA/tc_model.en
!mosesdecoder/scripts/recaser/train-truecaser.perl -corpus $HEMINGWAY_DATA/hemingway.lv.tok.txt -model $HEMINGWAY_DATA/tc_model.lv

!mosesdecoder/scripts/recaser/truecase.perl -model $HEMINGWAY_DATA/tc_model.en < $HEMINGWAY_DATA/hemingway.en.tok.txt > $HEMINGWAY_DATA/hemingway.en.tc.txt
!mosesdecoder/scripts/recaser/truecase.perl -model $HEMINGWAY_DATA/tc_model.lv < $HEMINGWAY_DATA/hemingway.lv.tok.txt > $HEMINGWAY_DATA/hemingway.lv.tc.txt

In [14]:
# !subword-nmt learn-joint-bpe-and-vocab --input en.tc.txt lv.tc.txt -s 10000 -o tokens.txt --write-vocabulary token_freq.en.txt token_freq.lv.txt
!mkdir -p $HEMINGWAY_DATA/bpe 
!subword-nmt learn-joint-bpe-and-vocab --input $HEMINGWAY_DATA/hemingway.en.tc.txt -s $num_bpe_merges -o $HEMINGWAY_DATA/bpe/tokens.en --write-vocabulary $HEMINGWAY_DATA/bpe/token_freq.en
!subword-nmt learn-joint-bpe-and-vocab --input $HEMINGWAY_DATA/hemingway.lv.tc.txt -s $num_bpe_merges -o $HEMINGWAY_DATA/bpe/tokens.lv --write-vocabulary $HEMINGWAY_DATA/bpe/token_freq.lv

no pair has frequency >= 2. Stopping
no pair has frequency >= 2. Stopping


In [15]:
def build_vocab(freq_file, vocab_size):
    vocab = ['<unk>', '<pad>', '<eos>']
    with open(freq_file, 'r') as f:
        for line in f.readlines():
            token, _ = line.split()
            vocab.append(token)

    return vocab[:vocab_size]

en_vocab = build_vocab(f'{HEMINGWAY_DATA}/bpe/token_freq.en', vocab_size)
lv_vocab = build_vocab(f'{HEMINGWAY_DATA}/bpe/token_freq.lv', vocab_size)

with open(f'{HEMINGWAY_DATA}/bpe/vocab.en', 'w') as f:
    for i, token in enumerate(en_vocab):
        f.write(f"{token} {i + 1} \n")

with open(f'{HEMINGWAY_DATA}/bpe/vocab.lv', 'w') as f:
    for i, token in enumerate(lv_vocab):
        f.write(f"{token} {i + 1} \n")

In [16]:
!subword-nmt apply-bpe -c $HEMINGWAY_DATA/bpe/tokens.en --vocabulary $HEMINGWAY_DATA/bpe/vocab.en --vocabulary-threshold 1 < $HEMINGWAY_DATA/hemingway.en.tc.txt > $HEMINGWAY_DATA/hemingway.en.BPE.txt
!subword-nmt apply-bpe -c $HEMINGWAY_DATA/bpe/tokens.lv --vocabulary $HEMINGWAY_DATA/bpe/vocab.lv --vocabulary-threshold 1 < $HEMINGWAY_DATA/hemingway.lv.tc.txt > $HEMINGWAY_DATA/hemingway.lv.BPE.txt

# !subword-nmt apply-bpe -c $HEMINGWAY_DATA/bpe/tokens.en --vocabulary $HEMINGWAY_DATA/bpe/token_freq.en --vocabulary-threshold 1 < $HEMINGWAY_DATA/hemingway.en.tc.txt > $HEMINGWAY_DATA/hemingway.en.BPE.txt
# !subword-nmt apply-bpe -c $HEMINGWAY_DATA/bpe/tokens.lv --vocabulary $HEMINGWAY_DATA/bpe/token_freq.lv --vocabulary-threshold 1 < $HEMINGWAY_DATA/hemingway.lv.tc.txt > $HEMINGWAY_DATA/hemingway.lv.BPE.txt

In [17]:
special_tokens = ['<unk>', '<pad>', '<eos>', '<sep>'] #, '<S>', '</S>', '<bos>', '<eos>', '<sep>', '<NONE>', '<|>']
                  
    #  '---Exits---']  #, COMMAND_TOKEN]

                 # '+open', '+closed', '+roasted', '+baked', '+fried', '+raw',
                 # '+sliced', '+diced', '+chopped', '++Carrying:', ]
normalizer = normalizers.Sequence([Strip(), Lowercase()])
pre_tokenizer = Whitespace()

model = tokenizers.models.WordLevel(unk_token='<unk>')
# model = tokenizers.models.WordPiece()
tokenizer = tokenizers.Tokenizer(model=model)


tokenizer.add_special_tokens(special_tokens)
tokenizer.normalizer = normalizer
tokenizer.pre_tokenizer = pre_tokenizer

# filelist = glob.glob(PTHRU_DIR+"valid/*.pthru")
# filelist.extend( glob.glob(PTHRU_DIR+"test/*.pthru"))
# filelist.extend( glob.glob(PTHRU_DIR+"train/*.pthru"))


# token_strs = [tok for (tok, span) in pre_tokenizer.pre_tokenize_str(str1)]
# print(token_strs)

# filelist = glob.glob(PTHRU_DIR+"valid/*.pthru")

filelist = glob.glob(f"{HEMINGWAY_DATA}/hemingway.*.BPE.txt")

filelist = sorted(filelist)
print(len(filelist), filelist[:10])


# unigram_trainer = tokenizers.trainers.UnigramTrainer()
# trainer = tokenizers.trainers.WordPieceTrainer(vocab_size=vocab_size)
trainer = tokenizers.trainers.WordLevelTrainer(vocab_size=joint_vocab_size, special_tokens=special_tokens)

tokenizer.train(files=filelist, trainer=trainer)

2 ['./data/hemingway/hemingway.en.BPE.txt', './data/hemingway/hemingway.lv.BPE.txt']


In [18]:
vocab_dict = tokenizer.get_vocab(with_added_tokens=False)
print("ACTUAL VOCAB SIZE =", len(vocab_dict))
print(vocab_dict)

ACTUAL VOCAB SIZE = 8730
{'stairs': 1244, 'neskatījās': 5888, 'tch': 8447, 'november': 4321, 'racing': 2823, 'durvīs': 5298, 'smoke': 3219, 'ians': 5037, 'translated': 6558, 'brother': 4166, 'smie': 6479, 'grabbed': 8232, 'majoram': 7174, 'nākamgad': 6081, 'ūsām': 5057, 'sniedza': 2569, 'stāsti': 3921, 'stāvot': 4658, 'employment': 8183, 'ways': 8279, 'ņu': 564, 'fire': 1524, 'either': 2721, 'varot': 4310, 'kasta': 6468, 'ative': 6535, 'no': 24, 'umu': 701, 'akmeņiem': 3966, 'kties': 2862, 'nošāva': 4073, 'spinner': 6699, 'ying': 6457, 'jūdzi': 4705, 'neatradu': 8292, 'cap': 985, 'dīti': 5841, 'deviņdesmit': 5539, 'sanitārs': 2638, 'small': 600, 'dzeršu': 4285, 'latin': 5597, 'ņas': 2075, 'iestrē': 6393, 'erno': 7923, 'heavy': 2428, 'ratos': 2872, 'pār': 157, 'via': 3165, 'skaitī': 4082, 'ik': 5117, 'wan': 8721, 'paša': 2600, 'salīdzinājumā': 8160, 'scr': 8656, 'otā': 4854, 'is': 58, 'ināju': 5361, 'avīzi': 3630, 'crowded': 3855, 'home': 740, 'large': 3575, 'iekārta': 5330, 'cepures': 

In [19]:
with open(f'{HEMINGWAY_DATA}/hemingway.lv.BPE.txt', 'r') as f:
    text_input = f.read()

with open(f'{HEMINGWAY_DATA}/hemingway.en.BPE.txt', 'r') as f:
    text_output = f.read()

#MinGPT

In [20]:
import random
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[:, [-1]]] = -float('Inf')
    return out

def calculate_attention_token(attention, top_k, model):
    logits = model.head(attention)
    logits = logits[:, -1, :]
    logits = top_k_logits(logits, top_k)

    probs = F.softmax(logits)

    _, ix = torch.topk(probs, k=1, dim=-1)
    ix = torch.multinomial(probs, num_samples=top_k)

    return ix[0]


@torch.no_grad()
def sample(model, x, steps, temperature=1.0, sample=False, top_k=None, output_attention=False):
    """
    take a conditioning sequence of indices in x (of shape (b,t)) and predict the next token in
    the sequence, feeding the predictions back into the model each time. Clearly the sampling
    has quadratic complexity unlike an RNN that is only linear, and has a finite context window
    of block_size, unlike an RNN that has an infinite context window.
    """
    block_size = model.get_block_size()
    model.eval()
    attention_state = [[] for _ in model.blocks]

    for k in range(steps):
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
        logits, _ = model(x_cond)
        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)
        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)

        if output_attention:
            b, t = x.size()

            for block_id in range(len(model.blocks)):
                att = model.blocks[block_id].attn.att
                attention_state[block_id].append(att)

        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)

    if output_attention:
        return x, attention_state

    return x


In [21]:
"""
GPT model:
- the initial stem consists of a combination of token encoding and a positional encoding
- the meat of it is a uniform sequence of Transformer blocks
    - each Transformer is a sequential combination of a 1-hidden-layer MLP block and a self-attention block
    - all blocks feed into a central residual pathway similar to resnets
- the final decoder is a linear projection into a vanilla Softmax classifier
"""

import math
import logging

import torch
import torch.nn as nn
from torch.nn import functional as F

logger = logging.getLogger(__name__)

class GPTConfig:
    """ base GPT config, params common to all GPT versions """
    embd_pdrop = 0.1
    resid_pdrop = 0.1
    attn_pdrop = 0.1

    def __init__(self, vocab_size, block_size, **kwargs):
        self.vocab_size = vocab_size
        self.block_size = block_size
        for k,v in kwargs.items():
            setattr(self, k, v)

class GPT1Config(GPTConfig):
    """ GPT-1 like network roughly 125M params """
    n_layer = 12
    n_head = 12
    n_embd = 768

class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_drop = nn.Dropout(config.attn_pdrop)
        self.resid_drop = nn.Dropout(config.resid_pdrop)
        # output projection
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.att = None

    def forward(self, x, layer_past=None):
        B, T, C = x.size()

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_drop(self.proj(y))

        self.att = att

        return y

class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.resid_pdrop),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class GPT(nn.Module):
    """  the full GPT language model, with a context size of block_size """

    def __init__(self, config):
        super().__init__()

        # input embedding stem
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.embd_pdrop)
        # transformer
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        # decoder head
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size
        self.apply(self._init_weights)

        logger.info("number of parameters: %e", sum(p.numel() for p in self.parameters()))

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self, train_config):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer

    def forward(self, idx, targets=None):
        b, t = idx.size()
        assert t <= self.block_size, "Cannot forward, model block size is exhausted."

        # forward the GPT model
        token_embeddings = self.tok_emb(idx) # each index maps to a (learnable) vector
        position_embeddings = self.pos_emb[:, :t, :] # each position maps to a (learnable) vector
        x = self.drop(token_embeddings + position_embeddings)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))

        return logits, loss


In [22]:
"""
Simple training loop; Boilerplate that could apply to any arbitrary neural network,
so nothing in this file really has anything to do with GPT specifically.
"""

import sacrebleu
import math
import logging
from random import choice

from tqdm import tqdm
import numpy as np

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.dataloader import DataLoader

logger = logging.getLogger(__name__)

def clean_tokens(sentence):
    return sentence.replace('@@ ', '').replace(' @', '').replace('@ ', '')

class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_tokens = 375e6 # these two numbers come from the GPT-3 paper, but may not be good defaults elsewhere
    final_tokens = 260e9 # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_path = None
    num_workers = 0 # for DataLoader

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

class Trainer:

    def __init__(self, model, train_dataset, test_dataset, valid_dataset, config):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.valid_dataset = valid_dataset
        self.config = config

        # take over whatever gpus are on the system
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)

    def save_checkpoint(self, postfix=''):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        checkpoint_path = self.config.ckpt_path + postfix + '.pt'
        logger.info("saving %s", checkpoint_path)
        torch.save(raw_model.state_dict(), checkpoint_path)

    def train(self):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset
            if split == 'test':
                data = self.test_dataset
            elif split == 'valid':
                data = self.valid_dataset
                model.eval()
            loader = DataLoader(data, shuffle=True, pin_memory=True,
                                batch_size=config.batch_size,
                                num_workers=config.num_workers)

            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            logits_total = None
            x_total = None
            y_total = None
            for it, (x, y) in pbar:

                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    logits, loss = model(x, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())
                    if split == 'valid':
                        if logits_total is None:
                            logits_total = logits
                            x_total = x
                            y_total = y
                        else:
                            logits_total = torch.cat((logits_total, logits), dim=0)
                            x_total = torch.cat((x_total, x), dim=0)
                            y_total = torch.cat((y_total, y), dim=0)
                        

                if is_train:
                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # number of tokens processed this step (i.e. label is not -100)
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens))
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. mean loss: {float(np.mean(losses)):.5f}. lr {lr:e}")

            if split == 'train':
                train_loss = float(np.mean(losses))
                print(f"train loss: {train_loss}")
                return train_loss

            if split == 'test':
                test_loss = float(np.mean(losses))
                print(f"test loss: {test_loss}")
                return test_loss

            if split == 'valid':
                test_loss = float(np.mean(losses))
                print(f"valid loss: {test_loss}")

                eval_results = []
                translation_results = []
                context_list = []

                for idx in range(len(logits_total)):
                    intent = (x_total[idx] == valid_dataset.tokenizer_input.encode(['<eos>'])[0]).nonzero(as_tuple=True)[0][0]

                    probs = F.softmax(logits_total[idx], dim=-1)
                    # sample from the distribution or take the most likely
                    _, predicted = torch.topk(probs, k=1, dim=-1)
                    context = clean_tokens(data.tokenizer_input.decode(x_total[idx][:intent - 1], True))
                    completion = clean_tokens(data.tokenizer_output.decode(predicted[intent:], True))
                    real = clean_tokens(data.tokenizer_output.decode(y_total[idx][intent:], True))

                    context_list.append(context)
                    translation_results.append(completion)
                    eval_results.append(real)
                
                with open('valid.txt', 'w') as f:
                    f.write("\n".join(translation_results))

                with open('eval.txt', 'w') as f:
                    f.write("\n".join(eval_results))

                with open('context.txt', 'w') as f:
                    f.write("\n".join(context_list))


                !cat valid.txt | mosesdecoder/scripts/tokenizer/detokenizer.perl -l lv > valid.detok.txt
                !cat eval.txt | mosesdecoder/scripts/tokenizer/detokenizer.perl -l lv > eval.detok.txt
                !cat context.txt | mosesdecoder/scripts/tokenizer/detokenizer.perl -l lv > context.detok.txt

                with open('eval.detok.txt', 'r') as f:
                    eval_results = [l.strip() for l in f.readlines()]
                with open('valid.detok.txt', 'r') as f:
                    translation_results = [l.strip() for l in f.readlines()]
                with open('context.detok.txt', 'r') as f:
                    context_list = [l.strip() for l in f.readlines()]

#                 idx = choice(range(len(context_list)))
                valid_sentences = ['the driver wore a cap and his face was thin and very tanned.',
                                   'outside it was getting dark.',
                                   'the two girls were asleep.',
                                   'I would like to have had the uniform off although I did not care much about the outward forms.',
                                   'I watched the flashes on San Gabriele.',
                                   'I asked.',
                                   '"no.']

                idx_list = [i for i, sentence in enumerate(eval_results) if sentence in valid_sentences]
                
                for idx in idx_list:
                    print(f'Input:            {context_list[idx]}')
                    print(f'Predicted output: {translation_results[idx]}')
                    print(f'Real output:      {eval_results[idx]}')
                    print('--------------------------------------------------')

                refs = [eval_results]
                sys = translation_results
                bleu = sacrebleu.corpus_bleu(sys, refs)
                print(f'BLEU: {bleu.score}')
                print('##############################################################')

                return test_loss, bleu.score

        train_loss_list = []
        test_loss_list = []
        valid_loss_list = []
        valid_bleu_list = []
        best_loss = float('inf')
        self.tokens = 0 # counter used for learning rate decay
        for epoch in range(config.max_epochs):

            train_loss = run_epoch('train')
            train_loss_list.append(train_loss)
            if self.test_dataset is not None:
                test_loss = run_epoch('test')
                test_loss_list.append(test_loss)

            if self.valid_dataset is not None:
                valid_loss, bleu_score = run_epoch('valid')
                valid_loss_list.append(valid_loss)
                valid_bleu_list.append(bleu_score)

            # supports early stopping based on the test loss, or just save always if no test set is provided
            good_model = self.test_dataset is None or test_loss < best_loss
            if self.config.ckpt_path is not None and good_model:
                best_loss = test_loss
                self.save_checkpoint("_best")

            if epoch % 10 == 0:
                self.save_checkpoint(f"_{epoch}")

            self.save_checkpoint("_last")

        return train_loss_list, test_loss_list, valid_loss_list, valid_bleu_list


#Training

In [23]:

class Tokenizer:
    def __init__(self, data, vocab_size, vocab):
        self.vocab_size = vocab_size
        self.vocab = vocab

        self.stoi = {ch: i for i, ch in enumerate(self.vocab)}
        self.itos = {i: ch for i, ch in enumerate(self.vocab)}
    
    def tokenize(self, data, block_size):
        tokenized_text = data.split()
        # Filter empty strings
        tokenized_text = [x for x in tokenized_text if x]
        result = []
        for tokenized in tokenized_text:
            # In case other single # found, replace them with <unk> special token, marking the element as unknown
            if tokenized in self.vocab:
                result.append(tokenized)
            else:
                result.append('<unk>')

        # in case the sentence is longer, than block_size, we trim the sentence
        return result[:block_size]
    
    def encode(self, data):
        return [self.stoi[s] for s in data]
    
    def decode(self, data, clean_paddings=False):
        text = ' '.join([self.itos[int(i)] for i in data])

        if not clean_paddings:
            return text
        return text.replace('<pad>', '').replace('  ', '')

In [24]:
# vocab_size = 10000

# vocab_input = None
# if os.path.exists('vocab_input.pkl'):
#     with open('vocab_input.pkl', 'rb') as f:
#         vocab_input = pickle.load(f)
        
# vocab_output = None
# if os.path.exists('vocab_output.pkl'):
#     with open('vocab_output.pkl', 'rb') as f:
#         vocab_output = pickle.load(f)

# building vocabluary can take some time. ~5 minutes for 10_000 tokens for each tokenizer. 
tokenizer_input = Tokenizer(text_input, vocab_size, lv_vocab)
tokenizer_output = Tokenizer(text_output, vocab_size, en_vocab)

In [25]:
# with open('vocab_input.pkl', 'wb') as f:
#     pickle.dump(tokenizer_input.vocab, f)

# with open('vocab_output.pkl', 'wb') as f:
#     pickle.dump(tokenizer_output.vocab, f)

In [26]:
# Shuffle texts by lines
texts = list(zip(text_output.splitlines(), text_input.splitlines()))
random.shuffle(texts)

output_texts, input_texts = zip(*texts)

In [27]:
# Split texts into train, test and validation datasets
train_dataset_size = round(0.75 * len(output_texts))
test_dataset_size = round(0.15 * len(output_texts))
valid_dataset_size = round(0.1 * len(output_texts))

train_input = input_texts[:train_dataset_size]
test_input = input_texts[train_dataset_size:train_dataset_size + test_dataset_size]
valid_input = input_texts[-valid_dataset_size:]

train_output = output_texts[:train_dataset_size]
test_output = output_texts[train_dataset_size:train_dataset_size + test_dataset_size]
valid_output = output_texts[-valid_dataset_size:]


In [28]:

with open('data/hemingway/train.lv', 'w') as f:
    f.write("\n".join(train_input))

with open('data/hemingway/test.lv', 'w') as f:
    f.write("\n".join(test_input))

with open('data/hemingway/valid.lv', 'w') as f:
    f.write("\n".join(valid_input))


with open('data/hemingway/train.en', 'w') as f:
    f.write("\n".join(train_output))

with open('data/hemingway/test.en', 'w') as f:
    f.write("\n".join(test_output))

with open('data/hemingway/valid.en', 'w') as f:
    f.write("\n".join(valid_output))


In [29]:
from torch.utils.data import Dataset

class WordDataset(Dataset):

    def __init__(self, output_text, input_text, tokenizer_output, tokenizer_input, block_size):
        self.tokenizer_output = tokenizer_output
        self.tokenizer_input = tokenizer_input

        self.block_size = block_size * 2 + 1
        self.output_text = [tokenizer_output.tokenize(t, block_size) for t in output_text]
        self.input_text = [tokenizer_input.tokenize(t, block_size) for t in input_text]

    def __len__(self):
        return len(self.output_text)

    def __getitem__(self, idx):
        """
        The idea is to get the input sentence
        and translate it to output sentence (sentences could be on any language).

        In the init method we already split a sentence into tokens and filled with spaces,
        to have an equal sentence size. In this method we just encode the tokens to
        ids (a list of numbers), and we're trying to map ids sequences
        """

        tokenized_input_text = self.tokenizer_input.encode(self.input_text[idx])
        tokenized_output_text = self.tokenizer_output.encode(self.output_text[idx])

        dix = tokenized_input_text + self.tokenizer_output.encode(['<eos>']) + tokenized_output_text
        if len(dix) < self.block_size:
            dix += self.tokenizer_output.encode(['<pad>']) * (self.block_size - len(dix))

        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        y[:len(tokenized_input_text) - 1] = -100

        return x, y

In [30]:
block_size = 100  # the estimate how long lines the text could be (token count)

train_dataset = WordDataset(train_output, train_input, tokenizer_output, tokenizer_input, block_size)
test_dataset = WordDataset(test_output, test_input, tokenizer_output, tokenizer_input, block_size)
valid_dataset = WordDataset(valid_output, valid_input, tokenizer_output, tokenizer_input, block_size)

In [31]:
number_of_heads = 8
number_of_layers = 6

# from mingpt.model import GPT, GPTConfig
embd_pdrop = 0.1
resid_pdrop = 0.1
attn_pdrop = 0.2

mconf = GPTConfig(tokenizer_output.vocab_size, train_dataset.block_size,
                  n_layer=number_of_layers, n_head=number_of_heads, n_embd=512,
                  embd_pdrop=embd_pdrop, resid_pdrop=resid_pdrop, attn_pdrop=attn_pdrop)

model = GPT(mconf)

In [32]:
# from mingpt.trainer import Trainer, TrainerConfig

tokens_per_epoch = len(train_dataset) * block_size
train_epochs = 100

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=train_epochs, batch_size=128, learning_rate=3e-4,
                      lr_decay=True, warmup_tokens=tokens_per_epoch, final_tokens=train_epochs*tokens_per_epoch,
                      ckpt_path='minGPT-LV-EN-translator_model.pt',
                      num_workers=1, weight_decay=0.0001, betas=(0.9, 0.98))
trainer = Trainer(model, train_dataset, test_dataset, valid_dataset, tconf)

In [33]:
param_count = sum([param.nelement() for param in model.parameters()])

print(f'Parameters count: {param_count}')

Parameters count: 24650240


In [34]:
train_loss_list, test_loss_list, valid_loss_list, valid_bleu_list = trainer.train()

  0%|          | 0/52 [00:00<?, ?it/s]/home/guntis/anaconda3/envs/tw131/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
epoch 1 iter 51: train loss 0.33015. mean loss: 0.76397. lr 2.999394e-04: 100%|██████████| 52/52 [00:22<00:00,  2.31it/s]

train loss: 0.7639719133193676


test loss: 0.32935057986866345
valid loss: 0.322789911712919
Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv
Input:            vaicāju
Predicted output: ..
Real output:      I asked.
--------------------------------------------------
Input:            labprāt būtu aizmetis šo uniformu, lai gan es daudz nebēdāju par ārieni
Predicted output: I the the the the the the <eos> the the the the the the the the the <eos> the the the the
Real output:      I would like to have had the uniform off although I did not care much about the outward forms.
--------------------------------------------------
Input:            es jautāju
Predicted output: I..
Real output:      I asked.
--------------------------------------------------
Input:            abas meitenes bija iemigušas
Predicted output: I <eos> the. the.
Real output:      the two girls were asleep.
------------------------------------------

epoch 2 iter 51: train loss 0.27574. mean loss: 0.31464. lr 2.994116e-04: 100%|██████████| 52/52 [00:20<00:00,  2.54it/s]

train loss: 0.3146414521795053


...

train loss: 0.00785995395675015


test loss: 0.35196403481743554
valid loss: 0.35328841635159086
Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv
Input:            es jautāju
Predicted output: I asked.
Real output:      I asked.
--------------------------------------------------
Input:            vaicāju
Predicted output: I asked.
Real output:      I asked.
--------------------------------------------------
Input:            metās jau tumsa
Predicted output: it it 's getting dark outside
Real output:      outside it was getting dark.
--------------------------------------------------
Input:            šoferim galvā bija cepure, un viņa seja bija kalsna un stipri iedegusi
Predicted output: the driver had the man, his face was dead and the good and
Real output:      the driver wore a cap and his face was thin and very tanned.
--------------------------------------------------
Input:            es jautāju
Predicted outp

epoch 100 iter 51: train loss 0.00942. mean loss: 0.00824. lr 2.933883e-04: 100%|██████████| 52/52 [00:21<00:00,  2.44it/s]

train loss: 0.00824137154715852


test loss: 0.358064892617139
valid loss: 0.3522955690111433
Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv
Input:            abas meitenes bija iemigušas
Predicted output: the girls girls were taken in
Real output:      the two girls were asleep.
--------------------------------------------------
Input:            nē
Predicted output: no no.
Real output:      "no.
--------------------------------------------------
Input:            es jautāju
Predicted output: I asked.
Real output:      I asked.
--------------------------------------------------
Input:            šoferim galvā bija cepure, un viņa seja bija kalsna un stipri iedegusi
Predicted output: the driver had the man, his hat was a and my good and
Real output:      the driver wore a cap and his face was thin and very tanned.
--------------------------------------------------
Input:            es jautāju
Predicted output: I as

In [35]:
epochs = range(len(test_loss_list))
# plt.subplots(nrows=number_of_layers, ncols=number_of_heads, figsize=(30, 20))
fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(20, 10))
axs[0].plot(epochs, train_loss_list)
axs[0].set_title('Train loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')

axs[0].plot(epochs, test_loss_list)
axs[0].set_title('Test loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')

axs[1].plot(epochs, valid_loss_list)
axs[1].set_title('Validation loss')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')

axs[2].plot(epochs, valid_bleu_list)
axs[2].set_title('Validation BLEU')
axs[2].set_xlabel('Epochs')
axs[2].set_ylabel('BLEU')

plt.show()

<ipython-input-35-6a4818c9122d>:24: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [36]:
plt.savefig("hemingway_losses.png")


#Evaluate

In [37]:
checkpoint = torch.load('minGPT-LV-EN-translator_model.pt_best.pt')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [38]:
train_loss_list

[0.7639719133193676,
 0.3146414521795053,
 0.27935900768408406,
 0.2568013006104873,
 0.23999384905283266,
 0.2252980378957895,
 0.21173224225640297,
 0.1975925060418936,
 0.18506930195368254,
 0.1715181154700426,
 0.1587147910434466,
 0.14599510010045308,
 0.13330758807177728,
 0.12219288787589623,
 0.11037113159321822,
 0.10023131393469296,
 0.0892990015160579,
 0.08032648551922578,
 0.07112763177316922,
 0.0629386598817431,
 0.05625098886398169,
 0.04923309822781728,
 0.04326917259738995,
 0.03824807194849619,
 0.033264896796586424,
 0.02919690103198473,
 0.025815418169188958,
 0.02320325872502648,
 0.020388654791391812,
 0.018424122862947676,
 0.016228069336368486,
 0.014917625257602105,
 0.013451874829255618,
 0.012232946661802439,
 0.0112497266023778,
 0.01029684727724928,
 0.00951353508907442,
 0.008740398800000548,
 0.00809917889105586,
 0.007646113007257764,
 0.007231812100284374,
 0.006878927502279671,
 0.006421974224324983,
 0.0062023886085416265,
 0.0059929796082612416,
 0.

In [39]:
with open('hemingway_train_loss.txt', 'w') as f:
    f.write('\n'.join([str(s) for s in train_loss_list]))


In [40]:
with open('hemingway_test_loss.txt', 'w') as f:
    f.write('\n'.join([str(s) for s in test_loss_list]))

with open('hemingway_valid_loss.txt', 'w') as f:
    f.write('\n'.join([str(s) for s in valid_loss_list]))

with open('hemingway_valid_blue.txt', 'w') as f:
    f.write('\n'.join([str(s) for s in valid_bleu_list]))

In [41]:
from random import choice

for _ in range(5):
    idx = choice(range(len(valid_output)))

    context = valid_input[idx]
    encoded_input = tokenizer_input.encode(tokenizer_input.tokenize(context, block_size))
    x = torch.tensor(encoded_input, dtype=torch.long)[None,...].to(trainer.device)
    y = sample(model, x, block_size, temperature=1.0, sample=False, top_k=10)[0]

    intent = len(encoded_input) + 1

    predicted = y[intent:]
    completion = tokenizer_output.decode(predicted, True)
    print(f'Input:            {context}')
    print(f'Predicted output: {completion}')
    print(f'Real output:      {valid_output[idx]}')
    print('--------------------------------------------------')

Input:            mēs varam apmesties augšā kalnos &quot; .
Predicted output: we were a little time . &quot;
Real output:      we can find some place up in the mountains . &quot;
--------------------------------------------------
Input:            &quot; esmu noguris no ie@@ šanas &quot; .
Predicted output: &quot; I &apos;m a little place . &quot; 
Real output:      &quot; I &apos;m tired of this walking / &apos; &quot; W@@ ell , all we have to do is walk now .
--------------------------------------------------
Input:            es attaisīju un iz@@ kr@@ at@@ ī@@ ju a@@ mu@@ le@@ tu
Predicted output: <eos> I &apos;ll go on my head and then I said . 
Real output:      I opened the capsule and spilled him out into my hand .
--------------------------------------------------
Input:            varbūt , tēvs .
Predicted output: maybe I said . 
Real output:      perhaps , father .
--------------------------------------------------
Input:            &quot; man ir daži liet@@ oti zo@@ be@@ ni 

In [42]:
idx = choice(range(len(valid_output)))

context = valid_input[idx]
encoded_input = tokenizer_input.encode(tokenizer_input.tokenize(context, block_size))
x = torch.tensor(encoded_input, dtype=torch.long)[None,...].to(trainer.device)
y, attention_state = sample(model, x, block_size, temperature=1.0, sample=False, top_k=10, output_attention=True)

intent = len(encoded_input) + 1

predicted = y[0][intent:]
completion = tokenizer_output.decode(predicted,)
print(f'Input:            {context}')
print(f'Predicted output: {completion}')
print(f'Real output:      {valid_output[idx]}')
print('--------------------------------------------------')


Input:            &quot; noteikti &quot; .
Predicted output: &quot; yes . &quot; <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Real output:      &quot; sure . &quot;
--------------------------------------------------


In [43]:
fig, plots = plt.subplots(nrows=number_of_layers, ncols=number_of_heads, figsize=(30, 20))

axis_text = tokenizer_input.decode(encoded_input, True).split()

axis_text.append('<eos>')

axis_text += tokenizer_input.decode(predicted, True).split()

limit = len(axis_text)
for bi in range(number_of_layers):
    for hi in range(number_of_heads):
        attetion_plot = torch.zeros(limit, limit)
        for di in range(limit):
            attetion_plot[:di, :di] = attention_state[bi][di][0,hi,:di,:di].data

        ax = plots[bi][hi]
        ax.matshow(attetion_plot.numpy(), cmap='bone')

        # Set up axes
        ax.set_xticklabels([''] + axis_text, rotation=90)
        ax.set_yticklabels([''] + axis_text)

        # Show label at every tick
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

        # Set up a title
        ax.set_title(f'Block {bi + 1} Head {hi + 1}', size=25, pad=30)
        
plt.show()

<ipython-input-43-5d85184c86e6>:20: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + axis_text, rotation=90)
<ipython-input-43-5d85184c86e6>:21: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + axis_text)
<ipython-input-43-5d85184c86e6>:30: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [44]:
# In case the previous cell is not plotting anything, uncomment the code below and execute. After that, the plotting should be fine.
# %matplotlib inline
# import numpy as np
# x = np.linspace(0, 10, 100)

# fig = plt.figure()
# plt.plot(x, np.sin(x), '-')
# plt.plot(x, np.cos(x), '--');

#Calculate BLEU

In [45]:
def clean_tokens(sentence):
    return sentence.replace('@@ ', '').replace(' @', '').replace('@ ', '')

In [46]:
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# smooth = SmoothingFunction().method7

translation_results = []
eval_text = []
bleu_results = []
for idx, context in enumerate(valid_input):
    encoded_input = tokenizer_input.encode(tokenizer_input.tokenize(context, block_size))
    x = torch.tensor(encoded_input, dtype=torch.long)[None,...].to(trainer.device)
    y = sample(model, x, block_size, temperature=1.0, sample=False, top_k=10)[0]

    intent = len(encoded_input) + 1
    predicted = y[intent:]
    completion = clean_tokens(tokenizer_output.decode(predicted, True))
    translation_results.append(completion)

    eval = clean_tokens(valid_output[idx])
    eval_text.append(eval)
    # bleu = sentence_bleu([eval], completion, smoothing_function=smooth)
    # bleu_results.append(bleu)

# print(f"Averare BLEU: {np.mean(bleu_results)}")

In [47]:
with open('hemingway_valid.out', 'w') as f:
    f.write("\n".join(translation_results))

with open('hemingway_valid.ref', 'w') as f:
    f.write("\n".join(eval_text))

In [48]:
!perl mosesdecoder/scripts/generic/multi-bleu.perl hemingway_valid.ref < hemingway_valid.out

BLEU = 7.13, 37.6/11.7/3.8/1.8 (BP=0.964, ratio=0.965, hyp_len=9173, ref_len=9509)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


In [49]:
!cat hemingway_valid.out | mosesdecoder/scripts/tokenizer/detokenizer.perl -l lv > hemingway_valid.detok.out
!cat hemingway_valid.ref | mosesdecoder/scripts/tokenizer/detokenizer.perl -l lv > hemingway_valid.detok.ref

Detokenizer Version $Revision: 4134 $
Language: lv
Detokenizer Version $Revision: 4134 $
Language: lv


In [50]:
#!pip install sacrebleu
!pip show sacrebleu

Name: sacrebleu
Version: 1.5.1
Summary: Hassle-free computation of shareable, comparable, and reproducible BLEU, chrF, and TER scores
Home-page: https://github.com/mjpost/sacrebleu
Author: Matt Post
Author-email: post@cs.jhu.edu
License: Apache License 2.0
Location: /home/guntis/anaconda3/envs/tw131/lib/python3.8/site-packages
Requires: portalocker
Required-by: 


In [51]:
import sacrebleu

with open('hemingway_valid.detok.ref', 'r') as f:
    eval_ref = [l.strip() for l in f.readlines()]
with open('hemingway_valid.detok.out', 'r') as f:
    translation_results = [l.strip() for l in f.readlines()]

refs = [eval_ref]
sys = translation_results
bleu = sacrebleu.corpus_bleu(sys, refs)
print(bleu.score)

7.127224006847502


#Interactive translator

In [52]:
context = input("Enter your English text to translate: ")

# Predict Latvian output
encoded_input = tokenizer_input.encode(tokenizer_input.tokenize(context, block_size))
x = torch.tensor(encoded_input, dtype=torch.long)[None,...].to(trainer.device)
y, attention_state = sample(model, x, block_size, temperature=1.0, sample=False, top_k=10, output_attention=True)

intent = len(encoded_input) + 1

predicted = y[0][intent:]
completion = tokenizer_output.decode(predicted, True)
print(f'Input:            {context}')
print(f'Predicted output: {completion}')


# Plot attention
fig, plots = plt.subplots(nrows=number_of_layers, ncols=number_of_heads, figsize=(30, 20))

axis_text = tokenizer_input.decode(encoded_input, True).split()

axis_text.append('<eos>')

axis_text += tokenizer_input.decode(predicted, True).split()

limit = len(axis_text)
for bi in range(number_of_layers):
    for hi in range(number_of_heads):
        attetion_plot = torch.zeros(limit, limit)
        for di in range(limit):
            attetion_plot[:di, :di] = attention_state[bi][di][0,hi,:di,:di].data

        ax = plots[bi][hi]
        ax.matshow(attetion_plot.numpy(), cmap='bone')

        # Set up axes
        ax.set_xticklabels([''] + axis_text, rotation=90)
        ax.set_yticklabels([''] + axis_text)

        # Show label at every tick
        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

        # Set up a title
        ax.set_title(f'Block {bi + 1} Head {hi + 1}', size=25, pad=30)
        
plt.show()

KeyboardInterrupt: Interrupted by user